# Results of the Analysis of Kickstarter Projects

- Gibt es Trends / Merkmale von erfolgreichen Projekten? Sollte man bestimmte Projektarten besonders fördern bzw. bewerben?
    - bereits gefunden: Trends in unterschiedlichen Produktkategorien --- Textchart
        - evtl. nochmal weiter unterteilen und schauen, ob es eine besonders erfolgreiche Subkategorie gibt
    - bereits gefunden: Trends in unterschiedlichen Preiskategorien (günstiger -> i.d.R. erfolgreicher) --- Tabelle
        - lässt sich in Kombination mit Produktkategorien und deren "besten Phasen" verdeutlichen --- Plot
        - spiegeln sich die Preiskategorien auch in den Produktkategorien wieder? Sind 
    - lassen sich die Projekte clustern? Findet man ein Cluster der besonders erfolgreichen wieder?

## TOC:
* [Introduction](#introduction): Detect Humans

---
<a id='introduction'></a>
## Step 1: Detect Humans


In [1]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

Second Bullet Header <a class="anchor" id="second-bullet"></a>